In [1]:
import pandas as pd
import base64
import os
import glob
from datetime import datetime
import numpy as np

In [2]:
df_original = pd.read_excel("../data/raw/Book1.xlsx")

In [3]:
df_plan = df_original[["a","b", "c", "d"]].dropna(subset=["a"])
df_plan.columns = ["Workout","Day","Length", "Serie"]

In [4]:
df_exercises = df_original[["Exercise","Sets","Reps","Kind","Weeks","Series"]]

In [5]:
def repetition(serie):
    df = df_plan[df_plan["Serie"]==serie]
    return pd.concat([df]*3 , ignore_index=True)

In [6]:
Weeks = pd.concat([repetition(1),repetition(2),repetition(3),repetition(4)]).reset_index(drop=True)
Weeks = Weeks.reset_index()
Weeks["index"] = Weeks["index"]+1

In [7]:
for i in range(1,13):
    filter_ = (Weeks["index"]>7*(i-1))&(Weeks["index"]<=7*i)
    Weeks.loc[filter_, "Week"] = "Week "+str(i)

In [8]:
df_main_table = pd.merge(Weeks,df_exercises, left_on=["Workout","Week"], right_on=["Kind","Weeks"], how="left")

In [9]:
df_final = df_main_table[["index","Workout","Day","Week","Exercise","Sets","Reps"]]

In [10]:
df_final.loc[df_final["Workout"]=="Intervals", "Exercise"] = "Cardio: 30-60-90 Intervals"

/opt/anaconda3/envs/rents/lib/python3.9/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [11]:
df_final.loc[df_final["Exercise"].str.contains("2A. Chinup/Lat Pulldown", na=False), "obs"] = "Use band for chinups as needed. Use Lat Pulldown if necessary."
df_final.loc[df_final["Exercise"].str.contains("1B. Dumbbell Stepups", na=False), "obs"] = "Step should be at 90-degree knee flexion. Complete all reps for one leg before switching."
df_final.loc[df_final["Exercise"].str.contains("2A. Incline DB Chest Press", na=False), "obs"] = "Set incline to 30 degrees."
df_final.loc[df_final["Exercise"].str.contains("1B. Dumbbell Split Squat", na=False), "obs"] = "Hold a dumbbell in each hand."
df_final.loc[df_final["Exercise"].str.contains("2B. Single Arm Cable Row", na=False), "obs"] = "Do not allow torso to rotate."
df_final.loc[df_final["Exercise"].str.contains("2C. Standing Zottman Curls", na=False), "obs"] = "Use rope attachment."
df_final.loc[df_final["Exercise"].str.contains("2. Trap Bar Deadlift", na=False), "obs"] = "One set on the minute, every minute."
df_final.loc[df_final["Exercise"].str.contains("2. Barbell Bench Press*", na=False), "obs"] = "One set on the minute, every minute."
df_final.loc[df_final["Exercise"].str.contains("2. Barbell Back Squat", na=False), "obs"] = "One set on the minute, every minute."

/opt/anaconda3/envs/rents/lib/python3.9/site-packages/pandas/core/indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)


In [12]:
df_final["Exercise"] = df_final["Exercise"].str.replace("\*","")

/var/folders/pl/wm7wjyvn1vvflk70yn1b6n6c0000gn/T/ipykernel_65201/3524762197.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df_final["Exercise"] = df_final["Exercise"].str.replace("\*","")
/var/folders/pl/wm7wjyvn1vvflk70yn1b6n6c0000gn/T/ipykernel_65201/3524762197.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final["Exercise"] = df_final["Exercise"].str.replace("\*","")


In [13]:
teste = df_final[df_final["Exercise"].str.contains("rowler",na=False)]["Exercise"].str.split(".",expand=True).drop_duplicates(1)


In [14]:
imgs = pd.read_excel("../data/raw/Book2.xlsx")
imgs['image'] = "<a href=" + imgs["image"] + " target='blank'><img src=" + imgs["image"] + " title='rent' width='150' height='150'/></a>"
imgs.iloc[0]['image']

"<a href=https://doctorwho.com.br/projects/img/01.gif target='blank'><img src=https://doctorwho.com.br/projects/img/01.gif title='rent' width='150' height='150'/></a>"

In [15]:
hipo = pd.merge(df_final,imgs,left_on="Exercise",right_on="EXERCISE",how="left")
hipo.loc[hipo["Exercise"].isna(), "Exercise"] = "Rest Day"
today = datetime.today()
hipo["Date"] = today
hipo["Date"] = pd.to_datetime(hipo["Date"])
temp = (hipo["index"]-1).apply(np.ceil).apply(lambda x: pd.Timedelta(x, unit="D"))
hipo["Date"] = hipo["Date"] + temp
hipo["Date"] = hipo["Date"].dt.strftime('%Y-%m-%d')
hipo

,index,Workout,Day,Week,Exercise,Sets,Reps,obs,EXERCISE,number,image,Date
0,1,Workout A,1.0,Week 1,1A. Goblet Squat,3.0,10,NaN,1A. Goblet Squat,1.0,<a href=https://doctorwho.com.br/projects/img/...,2021-12-26
1,1,Workout A,1.0,Week 1,2A. Chinup/Lat Pulldown,3.0,10,Use band for chinups as needed. Use Lat Pulldo...,2A. Chinup/Lat Pulldown,2.0,<a href=https://doctorwho.com.br/projects/img/...,2021-12-26
2,1,Workout A,1.0,Week 1,1B. Dumbbell Stepups,2.0,12,Step should be at 90-degree knee flexion. Comp...,1B. Dumbbell Stepups,3.0,<a href=https://doctorwho.com.br/projects/img/...,2021-12-26
3,1,Workout A,1.0,Week 1,2B. Single Arm Corner Press,2.0,12,NaN,2B. Single Arm Corner Press,4.0,<a href=https://doctorwho.com.br/projects/img/...,2021-12-26
4,1,Workout A,1.0,Week 1,1C. Prowler Chest Press,2.0,10,NaN,1C. Prowler Chest Press,5.0,<a href=https://doctorwho.com.br/projects/img/...,2021-12-26
...,...,...,...,...,...,...,...,...,...,...,...,...
340,83,Workout A,6.0,Week 12,3A. High-to-Low Cable Chest Flye,3.0,15,NaN,3A. High-to-Low Cable Chest Flye,52.0,<a href=https://doctorwho.com.br/projects/img/...,2022-03-18
341,83,Workout A,6.0,Week 12,1B. Chinup,4.0,8,NaN,1B. Chinup,53.0,<a href=https://doctorwho.com.br/projects/img/...,2022-03-18
342,83,Workout A,6.0,Week 12,2B. Seated Cable Row,4.0,8,NaN,2B. Seated Cable Row,54.0,<a href=https://doctorwho.com.br/projects/img/...,2022-03-18
343,83,Workout A,6.0,Week 12,3B. Straight Arm Pulldown,4.0,15,NaN,3B. Straight Arm Pulldown,55.0,<a href=https://doctorwho.com.br/projects/img/...,2022-03-18


In [16]:
ddf = hipo[hipo["Exercise"].str.contains("rowler")]
ddf.groupby("Week").count()

,index,Workout,Day,Exercise,Sets,Reps,obs,EXERCISE,number,image,Date
Week,,,,,,,,,,,
Week 1,4,4,4,4,4,4,0,4,4,4,4
Week 2,4,4,4,4,4,4,0,4,4,4,4
Week 3,4,4,4,4,4,4,0,4,4,4,4
Week 4,2,2,2,2,2,2,0,2,2,2,2
Week 5,2,2,2,2,2,2,0,2,2,2,2
Week 6,2,2,2,2,2,2,0,2,2,2,2
Week 7,2,2,2,2,2,2,0,2,2,2,2
Week 8,2,2,2,2,2,2,0,2,2,2,2
Week 9,2,2,2,2,2,2,0,2,2,2,2


In [17]:
ddf[ddf["Week"].str.contains("Week 1")]

,index,Workout,Day,Week,Exercise,Sets,Reps,obs,EXERCISE,number,image,Date
4,1,Workout A,1.0,Week 1,1C. Prowler Chest Press,2.0,10,NaN,1C. Prowler Chest Press,5.0,<a href=https://doctorwho.com.br/projects/img/...,2021-12-26
11,2,Workout B,2.0,Week 1,1C. Prowler Drive,2.0,D/B,NaN,1C. Prowler Drive,12.0,<a href=https://doctorwho.com.br/projects/img/...,2021-12-27
19,4,Workout A,4.0,Week 1,1C. Prowler Chest Press,2.0,10,NaN,1C. Prowler Chest Press,5.0,<a href=https://doctorwho.com.br/projects/img/...,2021-12-29
26,5,Workout B,5.0,Week 1,1C. Prowler Drive,2.0,D/B,NaN,1C. Prowler Drive,12.0,<a href=https://doctorwho.com.br/projects/img/...,2021-12-30


In [18]:
hipo.to_csv("../app/exercises.csv")